In [10]:
!pip install mysql-connector-python pandas
import pandas as pd
import mysql.connector as mysql


In [11]:
db_config = {
    "host": "20.105.3.31",
    "user": "HarishPandey",
    "password": "H@rish@321",
    "database": "outdoortribes"
}

In [16]:

# Connect to the database

conn = mysql.connect(**db_config)
cursor = conn.cursor()
cursor.execute("SET @startdate = DATE_FORMAT(CURDATE() - INTERVAL 1 MONTH, '%Y-%m-01');")
cursor.execute("SET @enddate = LAST_DAY(CURDATE() - INTERVAL 1 MONTH);")

In [41]:

# Connect to the database

conn = mysql.connect(**db_config)

# Create a cursor
# cursor.execute("set @startdate = DATE_FORMAT(CURDATE() - INTERVAL 1 MONTH, '%Y-%m-01');")
# cursor.execute("set @enddate LAST_DAY(CURDATE() - INTERVAL 1 MONTH) ;")

cursor = conn.cursor()
cursor.execute("SET @startdate = DATE_FORMAT(CURDATE() - INTERVAL 1 MONTH, '%Y-%m-01');")
cursor.execute("SET @enddate = LAST_DAY(CURDATE() - INTERVAL 1 MONTH);")


atv_band = """
WITH atv_band AS
(SELECT mobile, COUNT(DISTINCT uniquebillno) AS bills,
  SUM(amount) AS sales, SUM(amount) / COUNT(DISTINCT uniquebillno) AS ATV
FROM outdoortribes.txn_report_accrual_redemption WHERE txndate BETWEEN @startdate AND @enddate
 AND amount > 0 AND insertiondate <CURDATE() AND billno NOT LIKE '%test%' 
 AND billno NOT LIKE '%roll%' GROUP BY 1)
SELECT 
    CASE 
        WHEN ATV <= 1500 THEN 'Upto 1,500'
        WHEN ATV > 1500 AND ATV <= 3000 THEN '1,500-3,000' 
        WHEN ATV > 3000 AND ATV <= 4500 THEN '3,001-4,500'
        WHEN ATV > 4500 AND ATV <= 6000 THEN '4,501-6,000'
        WHEN ATV > 6000 AND ATV <= 7500 THEN '6,001-7,500'
        WHEN ATV > 7500 AND ATV <= 9000 THEN '7,501-9,000'
        WHEN ATV > 9000 AND ATV <= 10500 THEN '9,001-10,500'
        WHEN ATV > 10500 AND ATV <= 12000 THEN '10,501-12,000'
        WHEN ATV > 12000 THEN 'more than 12,000'
    END AS atv_band,
    COUNT(DISTINCT mobile) customers,
    SUM(bills) bills,
    SUM(sales) AS sales 
FROM atv_band 
GROUP BY 1 
ORDER BY 
    CASE atv_band
        WHEN 'Upto 1,500' THEN 1
        WHEN '1,500-3,000' THEN 2
        WHEN '3,001-4,500' THEN 3
        WHEN '4,501-6,000' THEN 4
        WHEN '6,001-7,500' THEN 5
        WHEN '7,501-9,000' THEN 6
        WHEN '9,001-10,500' THEN 7
        WHEN '10,501-12,000' THEN 8
        WHEN 'more than 12,000' THEN 9
    END;"""
cursor.execute(atv_band)
# Fetch data
result = cursor.fetchall()

df_atv=pd.DataFrame(result,columns=['atv_band', 'customers', 'bills','sales'])
print(df_atv.head())


cursor.execute("SET @startdate = DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 4 MONTH), '%Y-%m-01');")
cursor.execute("SET @enddate = LAST_DAY(DATE_SUB(CURDATE(), INTERVAL 1 MONTH));")
abv_mom ="""
SELECT 
    CASE
        WHEN PERIOD_DIFF = 3 THEN 'prev_third_month'
        WHEN PERIOD_DIFF = 2 THEN 'prev_second_month'
        WHEN PERIOD_DIFF = 1 THEN 'prev_month'
        WHEN PERIOD_DIFF = 0 THEN 'curr_month'
    END AS MONTHNAME,
    atv
FROM (
    SELECT 
        PERIOD_DIFF(DATE_FORMAT(@enddate, '%Y%m'), DATE_FORMAT(txndate, '%Y%m')) AS PERIOD_DIFF,
        SUM(amount)/COUNT(DISTINCT uniquebillno) AS atv
    FROM txn_report_accrual_redemption
    WHERE txndate BETWEEN @startdate AND @enddate
    GROUP BY PERIOD_DIFF(DATE_FORMAT(@enddate, '%Y%m'), DATE_FORMAT(txndate, '%Y%m'))
) AS sub
ORDER BY PERIOD_DIFF DESC;"""

cursor.execute(abv_mom)
# Fetch data
result = cursor.fetchall()
df_mom=pd.DataFrame(result,columns=['MONTHNAME', 'atv'])
print(df_mom.head())

cursor.execute("SET @startdate = DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 1 MONTH), '%Y-%m-01');")
cursor.execute("SET @enddate = LAST_DAY(@startdate);")


ab_kpis=""" 
SELECT 
    'total_loyalty_bills' AS TxnMonth,
    curr.total_loyalty_bills AS curr_month,
    prev_yr.total_loyalty_bills AS prev_yr_curr_month
FROM (
    SELECT SUM(bills) AS total_loyalty_bills
    FROM (
        SELECT COUNT(DISTINCT UniqueBillNo) AS bills
        FROM txn_report_accrual_redemption
        WHERE TxnDate BETWEEN @startdate AND @enddate
        AND storecode NOT LIKE '%demo%' 
        AND billno NOT LIKE '%test%' 
        AND billno NOT LIKE '%roll%'
        GROUP BY mobile
    ) a
) curr
CROSS JOIN (
    SELECT SUM(bills) AS total_loyalty_bills
    FROM (
        SELECT COUNT(DISTINCT UniqueBillNo) AS bills
        FROM txn_report_accrual_redemption
        WHERE TxnDate BETWEEN @startdate - INTERVAL 1 YEAR AND @enddate - INTERVAL 1 YEAR
        AND storecode NOT LIKE '%demo%' 
        AND billno NOT LIKE '%test%' 
        AND billno NOT LIKE '%roll%'
        GROUP BY mobile
    ) a
) prev_yr

UNION ALL

SELECT 
    'total_repeat_bills',
    curr.total_repeat_bills,
    prev_yr.total_repeat_bills
FROM (
    SELECT SUM(CASE WHEN maxf > 1 THEN bills END) AS total_repeat_bills
    FROM (
        SELECT 
            COUNT(DISTINCT UniqueBillNo) AS bills,
            MAX(frequencycount) AS maxf
        FROM txn_report_accrual_redemption
        WHERE TxnDate BETWEEN @startdate AND @enddate
        AND storecode NOT LIKE '%demo%' 
        AND billno NOT LIKE '%test%' 
        AND billno NOT LIKE '%roll%'
        GROUP BY mobile
    ) a
) curr
CROSS JOIN (
    SELECT SUM(CASE WHEN maxf > 1 THEN bills END) AS total_repeat_bills
    FROM (
        SELECT 
            COUNT(DISTINCT UniqueBillNo) AS bills,
            MAX(frequencycount) AS maxf
        FROM txn_report_accrual_redemption
        WHERE TxnDate BETWEEN @startdate - INTERVAL 1 YEAR AND @enddate - INTERVAL 1 YEAR
        AND storecode NOT LIKE '%demo%' 
        AND billno NOT LIKE '%test%' 
        AND billno NOT LIKE '%roll%'
        GROUP BY mobile
    ) a
) prev_yr

UNION ALL

SELECT 
    'total_loyalty_sales',
    curr.total_loyalty_sales,
    prev_yr.total_loyalty_sales
FROM (
    SELECT SUM(Amount) AS total_loyalty_sales
    FROM txn_report_accrual_redemption
    WHERE TxnDate BETWEEN @startdate AND @enddate
    AND storecode NOT LIKE '%demo%' 
    AND billno NOT LIKE '%test%' 
    AND billno NOT LIKE '%roll%'
) curr
CROSS JOIN (
    SELECT SUM(Amount) AS total_loyalty_sales
    FROM txn_report_accrual_redemption
    WHERE TxnDate BETWEEN @startdate - INTERVAL 1 YEAR AND @enddate - INTERVAL 1 YEAR
    AND storecode NOT LIKE '%demo%' 
    AND billno NOT LIKE '%test%' 
    AND billno NOT LIKE '%roll%'
) prev_yr

UNION ALL

SELECT 
    'total_repeat_sales',
    curr.total_repeat_sales,
    prev_yr.total_repeat_sales
FROM (
    SELECT SUM(CASE WHEN maxf > 1 THEN sales END) AS total_repeat_sales
    FROM (
        SELECT 
            SUM(Amount) AS sales,
            MAX(frequencycount) AS maxf
        FROM txn_report_accrual_redemption
        WHERE TxnDate BETWEEN @startdate AND @enddate
        AND storecode NOT LIKE '%demo%' 
        AND billno NOT LIKE '%test%' 
        AND billno NOT LIKE '%roll%'
        GROUP BY mobile
    ) a
) curr
CROSS JOIN (
    SELECT SUM(CASE WHEN maxf > 1 THEN sales END) AS total_repeat_sales
    FROM (
        SELECT 
            SUM(Amount) AS sales,
            MAX(frequencycount) AS maxf
        FROM txn_report_accrual_redemption
        WHERE TxnDate BETWEEN @startdate - INTERVAL 1 YEAR AND @enddate - INTERVAL 1 YEAR
        AND storecode NOT LIKE '%demo%' 
        AND billno NOT LIKE '%test%' 
        AND billno NOT LIKE '%roll%'
        GROUP BY mobile
    ) a
) prev_yr;"""

cursor.execute(ab_kpis)
# Fetch data
result = cursor.fetchall()
df_ab_kpis=pd.DataFrame(result,columns=['TxnMonth', 'curr_month','prev_yr_curr_month'])
print(df_ab_kpis.head())




cursor.execute("set @startdate= DATE_FORMAT(CURDATE() - INTERVAL 1 MONTH, '%Y-%m-01') ;")
cursor.execute("set @enddate= LAST_DAY(CURDATE() - INTERVAL 1 MONTH);")
cursor.execute("SET @startdate4m = DATE_SUB(@startdate, INTERVAL 3 MONTH);")
cursor.execute("SET @curr_month = MONTHNAME(@enddate);")
cursor.execute("SET @prev_month = MONTHNAME(DATE_SUB(@enddate, INTERVAL 1 MONTH));")
cursor.execute("SET @prev_second_month = MONTHNAME(DATE_SUB(@enddate, INTERVAL 2 MONTH));")
cursor.execute("SET @prev_third_month = MONTHNAME(DATE_SUB(@enddate, INTERVAL 3 MONTH));")


burn_earn = """ 
SELECT
    CASE 
        WHEN `month` = @curr_month THEN 'curr_month'
        WHEN `month` = @prev_month THEN 'prev_month'
        WHEN `month` = @prev_second_month THEN 'prev_second_month'
        WHEN `month` = @prev_third_month THEN 'prev_third_month'
        ELSE 'other_month' 
    END AS mom,
    SUM(points_collected) AS points_collected,
    SUM(points_reedemed) AS points_reedemed
FROM (
    SELECT
        MONTHNAME(txndate) AS `month`,
        SUM(pointscollected) AS points_collected,
        SUM(pointsspent) AS points_reedemed
    FROM txn_report_accrual_redemption 
    WHERE storecode <> 'demo' 
      AND billno NOT LIKE '%test%' 
      AND billno NOT LIKE '%roll%'
      AND txndate BETWEEN @startdate4m AND @enddate 
    GROUP BY 1
) a 
GROUP BY 1
ORDER BY 
    FIELD(mom, 'prev_third_month', 'prev_second_month', 'prev_month', 'curr_month'); """

cursor.execute(burn_earn)
# Fetch data
result = cursor.fetchall()
df_burn_earn=pd.DataFrame(result,columns=['MOM', 'points_collected','points_reedemed'])
print(df_burn_earn.head())

cursor.execute("set @startdate = DATE_FORMAT(CURDATE() - INTERVAL 1 MONTH, '%Y-%m-01');")
cursor.execute(" set @enddate = LAST_DAY(CURDATE() - INTERVAL 1 MONTH);")

burn_analysis = """
SELECT 
    YEAR(txndate) AS YEAR,
    SUM(pointscollected) AS points_collected,
    SUM(pointsspent) AS points_reedemed
FROM txn_report_accrual_redemption 
WHERE storecode <> 'demo' 
  AND billno NOT LIKE '%test%' 
  AND billno NOT LIKE '%roll%'
  AND txndate >='2024-01-01'
GROUP BY YEAR(txndate);
"""

cursor.execute(burn_analysis)
result = cursor.fetchall()
df_burn_analysis=pd.DataFrame(result,columns = ['year','points_collected','points_reedemed'])
print(df_burn_analysis.head())


cursor.execute("set @startdate = DATE_FORMAT(CURDATE() - INTERVAL 1 MONTH, '%Y-%m-01');")
cursor.execute("set @enddate = LAST_DAY(CURDATE() - INTERVAL 1 MONTH);")
cursor.execute("SET @startdate1m = DATE_SUB(@startdate, INTERVAL 1 MONTH);")
cursor.execute("SET @enddate1m = LAST_DAY(@startdate1m);")
cursor.execute("SET @startdate1y = DATE_SUB(@startdate, INTERVAL 1 YEAR);")
cursor.execute("SET @enddate1y = LAST_DAY(@startdate1y);")


coupon_data = """
  -- Main query
WITH monthly_data AS (
  SELECT 
    CASE 
      WHEN issueddate BETWEEN @startdate1m AND @enddate1m THEN 'prev_month'
      WHEN issueddate BETWEEN @startdate AND @enddate THEN 'curr_month' 
    END AS month_period,
    couponstatus,
    issuedmobile,
    couponoffercode,
    discount
  FROM coupon_offer_report
  WHERE 
    redeemedstorecode <> 'demo'
    AND issueddate BETWEEN @startdate1m AND @enddate  -- Dynamic date filter
)

SELECT 
  metric AS PERIOD,
  MAX(CASE WHEN month_period = 'prev_month' THEN VALUE END) AS prev_month,
  MAX(CASE WHEN month_period = 'curr_month' THEN VALUE END) AS curr_month
FROM (
  SELECT 'issued' AS metric, month_period, COUNT(*) AS VALUE
  FROM monthly_data
  GROUP BY month_period

  UNION ALL

  SELECT 'redeemers', month_period, COUNT(DISTINCT issuedmobile)
  FROM monthly_data
  WHERE couponstatus = 'Used'
  GROUP BY month_period

  UNION ALL

  SELECT 'coupons_redeemed', month_period, COUNT(couponoffercode)
  FROM monthly_data
  WHERE couponstatus = 'Used'
  GROUP BY month_period

  UNION ALL

  SELECT 'discount', month_period, SUM(discount)
  FROM monthly_data
  GROUP BY month_period
) AS metrics
GROUP BY metric
ORDER BY 
  CASE metric
    WHEN 'issued' THEN 1
    WHEN 'redeemers' THEN 2
    WHEN 'coupons_redeemed' THEN 3
    WHEN 'discount' THEN 4
  END;"""
cursor.execute(coupon_data)
result=cursor.fetchall()
df_coupon_data =pd.DataFrame(result,columns = ['period','prevs_month','curr_month'])
print(df_coupon_data.head())


cursor.execute("set @startdate = DATE_FORMAT(CURDATE() - INTERVAL 1 MONTH, '%Y-%m-01') ;")
cursor.execute("set @enddate = LAST_DAY(CURDATE() - INTERVAL 1 MONTH) ;")


store_wise_data = """ 
SELECT lpaasstore lpaas_store,storecode AS store_code,SUM(BILLS) bills,SUM(AMOUNT)amount,
 SUM(CUSTOMER)customers,SUM(nonloyaltybills) nonloyalty_bills,SUM(nonloyalty_sales)nonloyalty_sales
FROM (
SELECT 
lpaasstore, storecode,
SUM(amount) AS AMOUNT,COUNT(DISTINCT uniquebillno) AS BILLS,
 COUNT(DISTINCT mobile) AS CUSTOMER,0 AS nonloyaltybills,0 AS nonloyalty_sales
FROM txn_report_accrual_redemption a JOIN store_master b USING(storecode)
WHERE TxnDate BETWEEN @startdate AND @enddate AND modifiedbillno NOT LIKE '%test%'
AND storecode NOT LIKE '%demo%'AND modifiedbillno NOT LIKE '%roll%'
AND storecode NOT LIKE '%test%' GROUP BY  storecode
UNION ALL
SELECT lpaasstore,modifiedstorecode AS storecode,SUM(itemnetamount) AS AMOUNT,0 AS BILLS,0 AS CUSTOMER,
COUNT(DISTINCT uniquebillno) AS nonloyaltybills,SUM(itemnetamount) AS nonloyalty_sales
FROM sku_report_nonloyalty a JOIN store_master b ON a.modifiedstorecode = b.storecode
 WHERE modifiedTxnDate BETWEEN @startdate AND @enddate AND modifiedbillno NOT LIKE '%test%'
 AND a.storecode NOT LIKE '%demo%'AND a.modifiedbillno NOT LIKE '%roll%'
 AND a.storecode NOT LIKE '%test%'GROUP BY  modifiedstorecode
) AS combined GROUP BY  2 ORDER BY 2; """


cursor.execute(store_wise_data)
result = cursor.fetchall()
df_store_wise_data = pd.DataFrame(result,columns =['lpaas_store','store_code','bills','amount','customers','nonloyalty_bills','nonloyalty_sales'])
print(df_store_wise_data.head())

file_path = r"D:\\OneDrive - EasyRewardz Software Services Private Limited\\python\\blackberry\\chogori_mbr.xlsx"
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    df_atv.to_excel(writer, index=False, sheet_name='ATV Band')
    df_mom.to_excel(writer, index=False, sheet_name='ATV MoM')
    df_ab_kpis.to_excel(writer, index=False, sheet_name='AB_kpis')
    df_burn_earn.to_excel(writer, index=False, sheet_name='burn to earn')
    df_burn_analysis.to_excel(writer, index=False, sheet_name='burn anlaysis')
    df_coupon_data.to_excel(writer, index=False, sheet_name='coupon data')
    df_store_wise_data.to_excel(writer, index=False, sheet_name='store_wise_data')
    

cursor.close()
conn.close()
print("data export successfully to 'chogori_mbr_xlsx'")

      atv_band  customers bills       sales
0   Upto 1,500        221   256   333304.27
1  1,500-3,000        623   660  1594002.85
2  3,001-4,500        691   766  2995302.42
3  4,501-6,000        582   672  3630345.05
4  6,001-7,500        375   455  3180603.95
           MONTHNAME           atv
0   prev_third_month  10911.531381
1  prev_second_month  10171.284707
2         prev_month  10140.751756
3         curr_month  10293.132819
              TxnMonth   curr_month prev_yr_curr_month
0  total_loyalty_bills      6270.00            3947.00
1   total_repeat_bills      2740.00            1690.00
2  total_loyalty_sales  64512920.44        41991848.11
3   total_repeat_sales  26640136.04        17332212.36
                 MOM points_collected points_reedemed
0   prev_third_month       2187045.00       279303.00
1  prev_second_month       1652525.00       247880.00
2         prev_month       1710247.00       285634.00
3         curr_month       2928114.00       263214.00
   year points_c